In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from util import *
from entities import *
import yaml
from multiprocessing import Pool
from functools import partial
from random import seed
import sys
import os
from collections import Counter, defaultdict
from IPython.core.debugger import set_trace


In [8]:
yaml.warnings({'YAMLLoadWarning': False})

In [17]:
def model_runner(chunk_data,default_params_dict,n_replications, output_folder):

    chunk_num, chunk = chunk_data
    #turn_output_file = gzip.open("output/turn_output_{i}.tsv.gz".format(i=chunk_num),"w")
    turn_output_file = open(os.path.join(output_folder,"turn_output_{i}.tsv".format(i=chunk_num)),"w")
    #agent_output_file = gzip.open("output/agent_output_{i}.tsv.gz".format(i=chunk_num),"w")
    turn_output_leave_file = open(os.path.join(output_folder,"turn_output_leave_{i}.tsv".format(i=chunk_num)),"w")
    for i,row in enumerate(chunk):
        if i % 1000 == 0:
            print chunk_num, i
        for replication in range(n_replications):
            print "\t", replication
            #print replication
            params_dict = default_params_dict.copy()
            params_dict.update(row)
            params_dict['replication_number'] = replication
            params_dict['run_number'] = int(params_dict['run_number'])
            params_dict['turn_output_file'] = turn_output_file
            params_dict['turn_output_leave_file'] = turn_output_leave_file
            #params_dict['agent_output_file'] = agent_output_file
            run_single_model(params_dict)

    turn_output_file.close()
    turn_output_leave_file.close()
    #agent_output_file.close()




def run_single_model(params_dict):
    #params_dict['agent_output_file'] = agent_output_file
    P = ParameterHolder(params_dict)
    # How is agent competence/likeability determined?
    agent_promotability_fn = promotability_function_factory(P)
    # Functions for new agents
    initial_level_sex_fn = sex_function_factory(P, len(P.hierarchy_sizes)-1, 0)
    # How agents are promoted
    promotion_function = promotion_function_factory(P)
    # How agents decide to leave
    leave_function = leave_function_factory(P)
    assign_projects_fn = assign_projects_factory(P)
    agent_id = 0
    # Initialize the company
    company_hierarchy = []
    for level, level_size in enumerate(P.hierarchy_sizes):
        sex_fn = sex_function_factory(P, level, 0)
        l = []
        for _ in range(level_size):
            l.append(Agent(sex_function = sex_fn,
                            promotability_function=agent_promotability_fn,
                            time_of_creation=0,
                           id=agent_id))
            agent_id +=1
        company_hierarchy.append(l)

    # Okay, start the simulation

    # For each project cycle
    for turn in range(P.n_project_cycles):
        #### Print stats about the company
        print_stats(P, turn, company_hierarchy)

        # For each hierarchy level
        for level_index, company_level in enumerate(company_hierarchy):

            # compute the percentage of women at the level
            # for bias adjustments
            if P.project_bias_type == 'effect_size':
                if level_index == 0:
                    perc_male = (sum([agent.is_male for agent in company_level]) /
                                  float(len(company_level)))
                else:
                    lookup = company_hierarchy[level_index-1]
                    perc_male = (sum([agent.is_male for agent in lookup]) /
                                  float(len(lookup)))

                proj_success_fn = bias_function_factory(P,perc_male, 'success',turn)
                proj_failure_fn = bias_function_factory(P,perc_male, 'fail',turn)
                
            else:
                if P.project_bias_type =='threshold':
                    perc_women = (sum([not agent.is_male for agent in company_level]) /
                                  float(len(company_level)))
                elif P.project_bias_type == 'micro_macro':
                    if level_index == 0:
                        perc_women = 0.5
                    else:
                        lookup = company_hierarchy[level_index-1]
                        perc_women = (sum([not agent.is_male for agent in lookup]) /
                                  float(len(lookup)))
                # What happens on project success and failure?
                proj_success_fn = project_function_factory(P,perc_women, 'success')
                proj_failure_fn = project_function_factory(P,perc_women, 'fail')

            #### Assign Projects
            projects = assign_projects_fn(P, company_level, turn, level_index)
            #### Carry out the projects
            for project in projects:
                # Is the project successful? 1 for yes, 0 for no
                if project.is_successful:
                    for agent in project.agents:
                        agent.num_successful_projects += 1
                    proj_success_fn(project)
                else:
                    for agent in project.agents:
                        agent.num_failed_projects += 1
                    proj_failure_fn(project)
        # If its a promotion period
        if turn % P.projects_per_promotion_cycle == 0:

            ### Compute the turnover at the company
            for level_iter, company_level in enumerate(company_hierarchy):
                # See who leaves
                leaving_agents, staying_agents = leave_function(P,company_level,level)
                print_leave_stats(P,turn,leaving_agents,level_iter)
                # Remove them
                company_hierarchy[level_iter] = staying_agents

            ### Now, promote up, starting at the bottom
            for level_iter in range(len(company_hierarchy)-1):
                company_level = company_hierarchy[level_iter]
                    # find the top K at the level below me
                hire_from = company_hierarchy[level_iter + 1]
                current_level = len(company_hierarchy) - level_iter
                if P.promotion_intervention and turn >= P.promotion_intervention_span[0] and turn<=P.promotion_intervention_span[1]:
                    women_lowerbar = P.hierarchy_sizes[level_iter] * P.promotion_intervention_bar
                    women_lowerbar = int(ceil(women_lowerbar))
                    women_current = sum([ not i.is_male for i in company_level])
                    agents_to_hire,agents_remaining = promotion_function(hire_from,
                                                                          P.hierarchy_sizes[level_iter]-len(company_level),\
                                                                         n_women=women_lowerbar-women_current)
                else:
                     agents_to_hire,agents_remaining = promotion_function(hire_from,
                                                                          P.hierarchy_sizes[level_iter]-len(company_level))


                # promote them
                company_hierarchy[level_iter] = company_level + agents_to_hire
                # leave the remaining agents
                company_hierarchy[level_iter + 1] = agents_remaining     

            ### Add new agents to the bottom
            new_agents = []
            initial_level_sex_fn = sex_function_factory(P, len(P.hierarchy_sizes)-1, turn)
#             male = 0
#             all_people = 0
            for _ in range(P.hierarchy_sizes[-1] - len(company_hierarchy[-1])):
                new_agents.append(Agent(sex_function = initial_level_sex_fn,
                                        promotability_function=agent_promotability_fn,
                                        time_of_creation=turn,
                                        id = agent_id))
#                 if new_agents[-1].is_male:
#                      male += 1
#                 all_people += 1 
                agent_id +=1
#             print(male,all_people,male/all_people)
            company_hierarchy[-1] = company_hierarchy[-1] + new_agents


sys.argv = ['','minimal_nodownward.yaml',
            'default_params.yaml',
            'minimal_threshold_track_promotion_simple_leave',
            '100',
            '1',
            '14260']

if __name__ == "__main__":

    if len(sys.argv) != 6:
        print 'Usage: python model.py [path to experiment file] ',
        print '[path to default params file] [path to desired output folder] ',
        print ' [n_replications_per_condition] [n_cores] [random seed]'


    experiment_file, default_params_file, output_folder, n_replications, n_cores, rseed = sys.argv[1:]
    
#     default_params_dict = yaml.load(open(default_params_file))
#     leave = default_params_dict["leave_function_type"]
#     promo =default_params_dict["promotion_model"]
#     projAssign = default_params_dict["project_assignment_method"]
#     bias = default_params_dict["project_bias_type"]
    output_folder = 'UnfairStretchAssignment'#leave +'_'+ promo +'_' + projAssign+'_' + bias 
    
    seed(int(rseed))
    np.random.seed(int(rseed))

    n_replications = int(n_replications)
    n_cores = int(n_cores)

    experiment_details = yaml.load(open(experiment_file))

    print experiment_details
    experimental_runs = expand_grid(experiment_details)
    experimental_runs = experimental_runs.reset_index().rename(index=str,columns={"index":"run_number"})

    try:
        os.mkdir(output_folder)
    except:
        print "Not going to overwrite output!"
        #sys.exit(-1)

    experimental_runs.to_csv(os.path.join(output_folder, "experiment_details.csv"),index=False)
    experimental_runs = [x[1].to_dict() for x in experimental_runs.iterrows()]
    print(int(len(experimental_runs) / n_cores))
    chunked = list(enumerate(chunkify(experimental_runs, n_cores)))
    default_params_dict = yaml.load(open(default_params_file))
    runner_partial = partial(model_runner,
                             default_params_dict=default_params_dict,
                             n_replications=n_replications,
                             output_folder=output_folder)
    if n_cores > 1:
        p = Pool(n_cores)
        res = p.map(runner_partial, chunked)
        p.close()
        p.terminate()
    else:
        # run on single process
        for c in chunked:
            runner_partial(c)

Usage: python model.py [path to experiment file]  [path to default params file] [path to desired output folder]   [n_replications_per_condition] [n_cores] [random seed]
{'stretch_project_biased_bar': [1.2], 'project_min_men_stretch_project': [1], 'weight': [0.5], 'project_stretch_min_level_multiplier': [0.25], 'project_promotability_stretch_multiplier': [2.0], 'leave_function_type': ['simple'], 'promotion_intervention_span': [[168, 240]], 'project_women_mixed_group_percent_increase_failure': [1], 'promotion_intervention_norm': [0.4], 'promotion_intervention': [False], 'project_women_additional_promotability_percent_drop_on_complain': [1], 'stretch_intervention_bar': [1], 'project_promotability_boost_women_percent_of_men': [0.98], 'stretch_intervention_start': [0], 'external_norm': [0.01], 'project_min_women_stretch_project': [3], 'stretch_project_percentage': [0.1], 'stretch_project_biased_assignment': [True], 'stretch_intervention': [False], 'external_male_at_above_level': [0.7], 'pro

In [20]:
chunked[0][1]

[{'complaint_bias': 0.9,
  'external_male_at_above_level': 0.7,
  'external_norm': 0.01,
  'leave_function_type': 'simple',
  'mixed_effect_size': 0.01,
  'project_assignment_method': 'equalSoloGroupPromotability',
  'project_bias_type': 'effect_size',
  'project_min_men_stretch_project': 1,
  'project_min_women_stretch_project': 3,
  'project_promotability_boost_women_percent_of_men': 0.98,
  'project_promotability_drop_women_percent_of_men': 1.02,
  'project_promotability_stretch_multiplier': 2.0,
  'project_stretch_min_level_multiplier': 0.25,
  'project_turns_per_stretch': 5,
  'project_women_additional_promotability_percent_drop_on_complain': 1,
  'project_women_mixed_group_percent_drop_success': 1,
  'project_women_mixed_group_percent_increase_failure': 1,
  'project_women_percent_complain_on_mixed_success': 0.1,
  'promotability_function_type': 'simple',
  'promotion_intervention': False,
  'promotion_intervention_bar': 0.7,
  'promotion_intervention_norm': 0.4,
  'promotion_int